In [ ]:
import numpy as np
import heapq
from pylab import plot, show, bar
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import random
import pandas as pd #package for some uses and maybe for visualization

np.random.seed(0) # set the seed for the random numbers


In [ ]:
#####################################################
##### initiating arrival rates array, from csv ######
#####################################################
df = pd.read_csv('arrival_rates.csv')
df
arrival_rates= [
        [[],[]],
        [[],[],[]],
        [[],[],[]]
      ]

k = -1
for i in range(3):
    for j in range(len(arrival_rates[i])):
        k +=1 
        if k==8:
            break
        #print (k)
        opc= [df.loc[k, "other"], df.loc[k, "15--18"], df.loc[k, "7--10"]]
        arrival_rates[i][j].extend(opc)

arrival_rates[0].insert(0, None)

# TODO: perhaps change to other-to-other to zeros instead of none
print(arrival_rates)




In [17]:
########################################################
## Creating a dictionary with the range of each floor ##
# the range ressembles the location in the rate array ##
########################################################
lower_floor_range = {floor: 1 for floor in range(1,16)}
upper_floor_range = {floor: 2 for floor in range(16,26)}
lower_floor_range.update(upper_floor_range)
get_floor_range = lower_floor_range
get_floor_range[0] = 0 #ground floor

print(get_floor_range)

{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 0: 0}


In [20]:
########################
##### Event class ######
########################
class Event():
    def __init__(self, time, eventType, customer = -1, movie1 = -1):
        self.time = time # event time
        self.eventType = eventType # type of the event
        self.customer = customer
        self.movie1 = movie1
        heapq.heappush(P, self) #add the event to the events list

    def __lt__(self,event2):
        return self.time < event2.time

########################
#### Customer class ####
########################
class Customer(object):
    def __init__(self, start_floor, destination, arrival_time, i, elevator =-1):
        self.start_floor = start_floor
        self.destination = destination
        self.elevator = elevator
        self.patience = 15
        self.arrival_time = arrival_time
        self.id = i

    def is_elevator_exchange_needed(self): # TO CHANGE THAT LATER SO IT WILL USE THE get_floor_range DICTIONARY
        #checks if start_floor and destination are in the same floor range
        if (self.start_floor <= 15 and self.destination <= 15) or (self.start_floor in [0]+list(range(16,26)) and self.destination in [0]+list(range(16,26))):
            return False
        else:
            return True

########################
#### Elevator class ####
########################    
class Elevator(object):
    def __init__(self, name, current_floor):
        self.name = name
        self.current_floor = current_floor
        self.customers = []
        self.max_capacity = 15
        self.remaining_space = self.max_capacity - len(self.customers)


########################
## Low-Elevator class ##
######################## 
class Lower_floors_Elevator(Elevator):
    def __init__(self, name, current_floor):
        Elevator.__init__(self, name, current_floor)
        Elevator.floor_range = range(16)

#########################
## High-Elevator class ##
######################### 
class Upper_floors_Elevator(Elevator):
    def __init__(self, name, current_floor):
        Elevator.__init__(self, name, current_floor) # TODO: perhaps use super()
        Elevator.floor_range = [0] + list(range(16,26)) # TODO shouldnt it be elevator?


##########################
## returns current rate ##
##########################
def get_current_rate(start_floor, destination):
    # get current arrival rate according to start floor, 
    # destination, and curr_time(which would be determined
    # at the start of each While iteration)
    start_floor = get_floor_range[start_floor]
    destination = get_floor_range[destination]
    if curr_time >= 60 and curr_time < 4*60:
        #07:00-10:00
        hour_range = 2
    elif curr_time >= 9*60 and curr_time < 12*60:
        #15:00-18:00
        hour_range = 1
    else:
        #any other time
        hour_range = 0
    
    rate = arrival_rates[start_floor][destination][hour_range]
    #and now convert to exponential rate and return result:
    return 60/rate

# Setup and start the simulation
print('Spaceship Elevator')
curr_time = 0   
P = []
A = 0
SIM_TIME = 14*60 #simulation time in minutes
line = []


# Create the Elevators
Elevator1 = Lower_floors_Elevator("Elevator1",0)
Elevator2 = Lower_floors_Elevator("Elevator2",0)

Elevator3 = Upper_floors_Elevator("Elevator3",0)
Elevator4 = Upper_floors_Elevator("Elevator4",0)


i=0

x = np.random.exponential(60/7) # time for the first event 
Event(x ,"arriving")
    

while curr_time < SIM_TIME:
    
    event = heapq.heappop(P)
    curr_time = event.time
    print(curr_time)
    #curr_rate = get_current_rate(start_floor, destination)
    
    if event.eventType == "arriving" : 
        new_customer = Customer(theater, curr_time,i)
        movie = new_customer.movie
        num_tickets = new_customer.num_tickets
        if theater.available[movie] > 0 :
            i+=1
            theater.L+=1
            people_in.append(theater.L)
            time.append(curr_time)  
            if A==0:
                A = 1
                Event(curr_time +  1 ,"buy_tickets", new_customer) 
            else:
                heapq.heappush(line, (new_customer.arrival_time, new_customer))
            Event(curr_time + new_customer.patience ,"out_of_patience", new_customer) 
        Event(curr_time + np.random.exponential(1 / 2) ,"arriving") # create the next arrivel   
        

    elif event.eventType == "buy_tickets" : 
        if event.customer.left == False: #check if the customer left the queue
            movie = event.customer.movie
            # Check if enough tickets left.
            if theater.available[movie] < event.customer.num_tickets:
                # customer leaves after some discussion
                Event(curr_time + 0.5 ,"argue_with_seller", event.customer)
            else:      # Buy tickets
                theater.available[movie] -= num_tickets
                if theater.available[movie] < 1:
                    # Trigger the "sold out" event for the movie
                    print("HERE MAYBE")
                    Event(curr_time ,"sold_out", movie1 = movie )
            theater.L -=1
            people_in.append(theater.L)
            time.append(curr_time)
            event.customer.left = True
        if line:
            arrival_time, customer = heapq.heappop(line)
            Event(curr_time +  1 ,"buy_tickets", customer)
        else:
            A = 0
            
    elif event.eventType == "argue_with_seller" : 
        if line:
            arrival_time, customer = heapq.heappop(line)
            Event(curr_time +  1 ,"buy_tickets", customer)
        else:
            A = 0

          
    elif event.eventType == "sold_out" : 
        print("IM HERE")
        theater.when_sold_out[movie] = curr_time
        theater.available[movie] = 0
        line = [item for item in line if item[1].movie != event.movie]
        if not any([movie==None for movie in theater.when_sold_out.values()]):
            theater.all_sold_out = True
        


        
    elif event.eventType == "out_of_patience" : 
        if event.customer.left != True:
            event.customer.left = True
            theater.num_leavers[movie] += 1
            theater.L -=1
            people_in.append(theater.L)
            time.append(curr_time)
        

'''
# Analysis/results
for movie in theater.movies:
    print('Movie "%s" sold out %.1f minutes after ticket counter '
          'opening.' % (movie, theater.when_sold_out[movie]))
    print('  Number of people leaving queue: %s' %
          theater.num_leavers[movie])

'''

Spaceship Elevator


AttributeError: 'Lower_floors_Elevator' object has no attribute 'max_capacity'

In [ ]:
if (8,5) < (8,2):
    print("yes")
else:
    print("no")

In [ ]:
a= 22
b= 17

if (a <= 15 and b <= 15) or (a in [0]+list(range(16,26)) and b in [0]+list(range(16,26))):
    print("dude")
else:
    print("jerk")


In [ ]:
#TEST FOR FUNCTION get_current_rate
curr_time = 67.45

def get_current_rate(start_floor, destination):
    start_floor = get_floor_range[start_floor]
    destination = get_floor_range[destination]
    if curr_time >= 60 and curr_time < 240:
        #07:00-10:00
        hour_range = 2
    elif curr_time >= 540 and curr_time < 720:
        #15:00-18:00
        hour_range = 1
    else:
        #any other time
        hour_range = 0
    
    return arrival_rates[start_floor][destination][hour_range]


print (get_current_rate(12,18))

In [ ]:
#TEST FOR building arrival rates

df = pd.read_csv('arrival_rates.csv')
df
LPL = [
        [[],[]],
        [[],[],[]],
        [[],[],[]]
      ]
k = -1
for i in range(3):
    for j in range(len(LPL[i])):
        k +=1 
        if k==8:
            break
        #print (k)
        opc= [df.loc[k, "other"], df.loc[k, "15--18"], df.loc[k, "7--10"]]
        LPL[i][j].extend(opc)

LPL[0].insert(0, None)
#df.iterrows()

#for index, row in df.iterrows():
 #   print(row['7--10'], row['15--18'], row["other"])

print(LPL[1][0])